# S1W2D4：PyTorch 建模基石：`nn.Module`

**今日目标：**

1.  理解 `nn.Module` 作为所有模型的“父类”的核心地位。
2.  **[重中之重]** 熟练掌握 `__init__` 和 `forward` 两个函数的不同分工。
3.  学会使用 `nn.Linear` 和 `nn.ReLU` 这两个最基础的“积木”来搭建一个简单的网络。

## 什么是 `nn.Module`？(“积木的蓝图”)

`nn.Module` 是 `torch.nn` 库提供的一个基类（父类）。在 PyTorch 中，你**所有**的神经网络模型（无论是简单的全连接网络，还是未来复杂的 Transformer），都**必须**继承 (inherit) 它。

**为什么要继承 `nn.Module`？**

因为它就像一个“智能管家”，在你搭建模型时，它在幕后自动帮你处理了大量复杂工作：

  * **追踪参数：** 它会自动“注册”并管理你模型中所有可学习的参数（即 `Tensor` 形式的 `weight` 和 `bias`）。你不需要再手动管理 `requires_grad=True`，`nn.Module` 内部的“层”（如 `nn.Linear`）会自动设置好。
  * **管理状态：** 它可以帮你一键切换模型的“训练模式” (`model.train()`) 和“评估模式” (`model.eval()`)。这在未来使用 Dropout 和 BatchNorm 等层时至关重要。
  * **嵌套模型：** 它允许你像套娃一样，把小的 `nn.Module` 组合成一个大的 `nn.Module`，非常灵活。

## 模型的“二元结构”：`__init__` 与 `forward` (面试必考)

继承 `nn.Module` 后，你必须重写 (override) 它的两个核心方法。这是面试官判断你是否真正理解 PyTorch 的关键。

**1. `__init__(self)` (构造函数 - “声明层”)**

  * **职责：** 这是你模型的“购物清单”或“零件盒”。你在这里**声明**并**初始化**你这个模型**将会用到**的所有“积木”（即各种层）。
  * **规则：**
    1.  第一行必须是 `super().__init__()`，用来调用父类 `nn.Module` 的初始化函数，这是“标准开场白”。
    2.  在这里，你把你需要的层定义为**类的属性**（即 `self.xxx`），例如 `self.layer1 = nn.Linear(10, 20)`。
  * **注意：** 在 `__init__` 中，数据**不**流动。你只是在“准备零件”。

**2. `forward(self, x)` (前向传播 - “定义数据流”)**

  * **职责：** 这是你模型的“装配说明书”。你在这里**定义**数据 `x` 是如何**流过**你在 `__init__` 中声明的那些层的。
  * **规则：**
    1.  它接收一个（或多个）`Tensor` `x` 作为输入。
    2.  你在这里**调用**（call）你在 `__init__` 中定义的层，并把数据传递给它们。例如 `x = self.layer1(x)`。
    3.  它必须 `return` 一个（或多个）`Tensor` 作为模型的输出。
  * **注意：** PyTorch 有个“魔法”：你**永远不要**手动调用 `model.forward(x)`。你只需要像调用函数一样调用你的模型实例 `model(x)`，PyTorch 会自动替你调用 `forward` 方法。

## 第一个“积木”：`nn.Linear` 和 `nn.ReLU`

1.  **`nn.Linear(in_features, out_features)` (全连接层)**

      * **作用：** 执行一个 $y = xW^T + b$ 的线性变换。
      * `in_features`：输入特征的数量（即上一层神经元的数量）。
      * `out_features`：输出特征的数量（即这一层神经元的数量）。
      * 当你定义它时，PyTorch 会自动帮你创建 `weight` (权重) 和 `bias` (偏置) 两个 `Tensor` 作为可学习参数。

2.  **`nn.ReLU()` (激活函数层)**

      * **作用：** 执行 $f(x) = \max(0, x)$ 的非线性激活。
      * **注意：** `nn.ReLU()` 是一个“层”模块，它没有可学习的参数（`weight` 或 `bias`）。
      * **[提示]** PyTorch 还提供了函数式的 `torch.nn.functional.relu()`（通常简写为 `F.relu`），它和 `nn.ReLU()` 效果一样，只是用法不同。

## 代码实践：搭建你的第一个神经网络

**目标：** 我们来搭建一个简单的多层感知机 (MLP)，用于处理 MNIST 数据集（$28 \times 28 = 784$ 个输入像素，10 个输出类别）。

  * **输入层：** 784 个特征
  * **隐藏层：** 128 个神经元，使用 ReLU 激活
  * **输出层：** 10 个特征（对应 0-9 十个数字的得分）

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F # 通常F被用于调用无参数的函数式API

# 1. 定义我们自己的模型类，它必须继承 nn.Module
class MyMLP(nn.Module):
    
    # 2. [__init__]: 声明模型将用到的所有“层”
    def __init__(self):
        super(MyMLP, self).__init__() # 必须的“开场白”
        
        # 声明第一个全连接层 (in_features=784, out_features=128)
        self.fc1 = nn.Linear(784, 128) 
        
        # 声明第二个全连接层 (in_features=128, out_features=10)
        # 注意：这一层的 in_features 必须等于上一层的 out_features
        self.fc2 = nn.Linear(128, 10)
        
        # 我们也可以在这里声明激活函数层
        # self.relu = nn.ReLU() 
        # (但我们待会儿在 forward 中将使用 F.relu() 来展示另一种方式)

    # 3. [forward]: 定义数据如何流过这些层
    def forward(self, x):
        # x 的输入形状: [batch_size, 784]
        
        # 数据流过第一个全连接层
        x = self.fc1(x) # 形状变为 [batch_size, 128]
        
        # 应用 ReLU 激活函数
        # 方式1: 使用在 __init__ 中定义的 nn.ReLU()
        # x = self.relu(x) 
        # 方式2: 使用 functional API (更常见，因为它没有参数)
        x = F.relu(x)
        
        # 数据流过第二个全连接层
        x = self.fc2(x) # 形状变为 [batch_size, 10]
        
        # 输出层通常不加激活，因为 nn.CrossEntropyLoss 会帮我们处理
        return x

# --- 下面是测试代码 ---

# 4. 实例化你的模型
# 这会调用 MyMLP 的 __init__ 方法
model = MyMLP()

# 5. 打印模型结构
# 你会清晰地看到你在 __init__ 中声明的层和它们的参数
print("--- 模型结构 ---")
print(model)
print("----------------")

# 6. 创建一个“假”的输入数据 (模拟一个批次)
# batch_size=64, 特征=784
dummy_input = torch.randn(64, 784) 

# 7. 将数据喂给模型
# 这会自动调用 MyMLP 的 forward 方法
output = model(dummy_input)

# 8. 检查输出形状
print(f"输入数据的形状: {dummy_input.shape}")
print(f"模型输出的形状: {output.shape}")

# 检查：输出形状必须是 [64, 10]
assert output.shape == (64, 10)
print("\n代码运行成功！你已经掌握了 nn.Module 的基本结构。")

--- 模型结构 ---
MyMLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
----------------
输入数据的形状: torch.Size([64, 784])
模型输出的形状: torch.Size([64, 10])

代码运行成功！你已经掌握了 nn.Module 的基本结构。


## **W2D4 今日行动清单**

1.  **✅ 运行代码：** 亲手将上面的代码在你的环境中运行一遍，确保你看到了打印出的模型结构和正确的输出形状。
2.  **✅ 理解分工：** 指着你代码中的 `__init__` 和 `forward`，大声告诉自己它们分别在做什么（“这个是声明”、“这个是运行”）。
3.  **✅ 检查输出：** 仔细看 `print(model)` 的输出结果，它是不是只显示了 `fc1` 和 `fc2`？PyTorch 就是通过 `__init__` 中的 `self.xxx = ...` 来“发现”这些层的。
4.  **✅ 思考题 (面试模拟)：**
      * 在上面的 `forward` 方法中，我们定义了数据流。如果在 `__init__` 方法中（比如在 `self.fc2` 后面）也写一行 `x = self.fc1(x)`，会发生什么？为什么？
      * 反过来，如果我不在 `__init__` 中定义 `self.fc1`，而是直接在 `forward` 方法里写 `fc1 = nn.Linear(784, 128)`，然后再 `x = fc1(x)`，这样做（虽然语法可能通过）会导致什么严重问题？

等你完成了今天的实践，可以告诉我你对最后那个思考题的答案。这能极好地检验你是否真正理解了 `__init__` 和 `forward` 的分工。